In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
movies_df = pd.read_json('movies_uptodate_with_old_ratings.json')

In [4]:
movies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,popularity,vote_average,vote_count
0,False,Heat Collection,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",https://www.20thcenturystudios.com/movies/heat,949,tt0113277,en,Heat,Obsessive master thief Neil McCauley leads a t...,...,187436818,170,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A Los Angeles crime saga.,Heat,False,17.924927,7.7,1886
1,False,James Bond Collection,58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.mgm.com/movies/goldeneye,710,tt0113189,en,GoldenEye,When a powerful satellite system falls into th...,...,352194034,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No limits. No fears. No substitutes.,GoldenEye,False,14.686036,6.6,1194
2,False,None,98000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,10017322,124,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,7.284477,5.7,137
3,False,None,50000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",,524,tt0112641,en,Casino,"In early-1970s Las Vegas, Sam ""Ace"" Rothstein ...",...,116112375,179,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No one stays at the top forever.,Casino,False,10.137389,7.8,1343
4,False,None,16000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,4584,tt0114388,en,Sense and Sensibility,"The Dashwood sisters, sensible Elinor and pass...",...,134582776,136,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,10.673167,7.2,364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,False,Laskar Pelangi Collection,0,"[{'id': 18, 'name': 'Drama'}]",,80831,tt1576459,id,Sang Pemimpi,A trio of friends struggles to make it through...,...,0,121,"[{'english_name': 'Indonesian', 'iso_639_1': '...",Released,,The Dreamer,False,0.083371,6.5,2
2818,False,Frankenstein (Hammer) Collection,0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",,3104,tt0061683,en,Frankenstein Created Woman,A deformed tormented girl drowns herself after...,...,0,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Now Frankenstein has created a beautiful woman...,Frankenstein Created Woman,False,2.302582,5.9,33
2819,False,None,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://studioslon.com/film6.html,64197,tt1047845,ru,Путешествие с домашними животными,Plucked from an orphanage as a literal love sl...,...,0,97,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Travelling with Pets,False,0.528657,6.0,5
2820,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,98604,tt2147597,ru,Zолушка,"Masha Krapivina - is yet beautiful, and not th...",...,5169134,91,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Cinderella,False,0.803588,4.6,6


In [5]:
movies_id = movies_df['id'].copy()

In [6]:
import requests
import time
from tqdm import tqdm  # Import the tqdm library

API_URL_BEFORE = 'https://api.themoviedb.org/3/movie/' 
API_URL_AFTER = '?api_key=e9e9d8da18ae29fc430845952232787c&'
requests_per_second = 50
connections_per_ip = 20
dfs = []
# Calculate the delay between requests to respect the rate limit
delay_between_requests = 1 / requests_per_second

# Use tqdm to create a progress bar for the loop
for movie_id in tqdm(movies_id, desc="Processing Movies", unit="movie"):
    # Introduce a delay between requests
    time.sleep(delay_between_requests)
        
    response = requests.get(API_URL_BEFORE + str(movie_id) + API_URL_AFTER)

    if response.status_code == 200:
        json_data = response.json()

        # Extract relevant information from the JSON data

        relevant_data = {
            'backdrop_path': json_data.get('backdrop_path'),
        }

        df = pd.DataFrame([relevant_data])  # Create DataFrame from the relevant data
        dfs.append(df)
    elif response.status_code == 404:
        print(f"Movie with ID {movie_id} not found. Skipping...")
    else:
        print(f"Error in request for movie {movie_id}: {response.status_code}")

# Concatenate all DataFrames into a single one
final_df = pd.concat(dfs, ignore_index=True)

# Now you can work with the final DataFrame
print(final_df.head())

Processing Movies: 100%|██████████| 2822/2822 [06:50<00:00,  6.87movie/s]


                      backdrop_path
0   /X7zKxmyrVmYCfcyvDgVLH8iNzA.jpg
1  /5jVWJiPpmk216Q7vjjlwNXXE6ee.jpg
2  /yhLeQvc841cBcXHgqDZmJpflDxG.jpg
3  /pLR2O3dmA9xkCiPA26U7ErCUYSi.jpg
4  /mZfkW31Y9a8DvISpO9nj5Tc7neo.jpg


In [8]:
final_df

,backdrop_path
0,/X7zKxmyrVmYCfcyvDgVLH8iNzA.jpg
1,/5jVWJiPpmk216Q7vjjlwNXXE6ee.jpg
2,/yhLeQvc841cBcXHgqDZmJpflDxG.jpg
3,/pLR2O3dmA9xkCiPA26U7ErCUYSi.jpg
4,/mZfkW31Y9a8DvISpO9nj5Tc7neo.jpg
...,...
2817,/iFXIQgddpVbc76L1xg6rtOIbJ7A.jpg
2818,/6R9eCdkYrYIwHjGfqm8bdLUQV9Y.jpg
2819,None
2820,None


In [9]:
movies_df['backdrop_path'] = final_df['backdrop_path']

In [10]:
movies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,popularity,vote_average,vote_count,backdrop_path
0,False,Heat Collection,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",https://www.20thcenturystudios.com/movies/heat,949,tt0113277,en,Heat,Obsessive master thief Neil McCauley leads a t...,...,170,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A Los Angeles crime saga.,Heat,False,17.924927,7.7,1886,/X7zKxmyrVmYCfcyvDgVLH8iNzA.jpg
1,False,James Bond Collection,58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.mgm.com/movies/goldeneye,710,tt0113189,en,GoldenEye,When a powerful satellite system falls into th...,...,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No limits. No fears. No substitutes.,GoldenEye,False,14.686036,6.6,1194,/5jVWJiPpmk216Q7vjjlwNXXE6ee.jpg
2,False,None,98000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,124,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,7.284477,5.7,137,/yhLeQvc841cBcXHgqDZmJpflDxG.jpg
3,False,None,50000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",,524,tt0112641,en,Casino,"In early-1970s Las Vegas, Sam ""Ace"" Rothstein ...",...,179,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,No one stays at the top forever.,Casino,False,10.137389,7.8,1343,/pLR2O3dmA9xkCiPA26U7ErCUYSi.jpg
4,False,None,16000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,4584,tt0114388,en,Sense and Sensibility,"The Dashwood sisters, sensible Elinor and pass...",...,136,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Lose your heart and come to your senses.,Sense and Sensibility,False,10.673167,7.2,364,/mZfkW31Y9a8DvISpO9nj5Tc7neo.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,False,Laskar Pelangi Collection,0,"[{'id': 18, 'name': 'Drama'}]",,80831,tt1576459,id,Sang Pemimpi,A trio of friends struggles to make it through...,...,121,"[{'english_name': 'Indonesian', 'iso_639_1': '...",Released,,The Dreamer,False,0.083371,6.5,2,/iFXIQgddpVbc76L1xg6rtOIbJ7A.jpg
2818,False,Frankenstein (Hammer) Collection,0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",,3104,tt0061683,en,Frankenstein Created Woman,A deformed tormented girl drowns herself after...,...,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Now Frankenstein has created a beautiful woman...,Frankenstein Created Woman,False,2.302582,5.9,33,/6R9eCdkYrYIwHjGfqm8bdLUQV9Y.jpg
2819,False,None,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://studioslon.com/film6.html,64197,tt1047845,ru,Путешествие с домашними животными,Plucked from an orphanage as a literal love sl...,...,97,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Travelling with Pets,False,0.528657,6.0,5,None
2820,False,None,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",,98604,tt2147597,ru,Zолушка,"Masha Krapivina - is yet beautiful, and not th...",...,91,"[{'english_name': 'Russian', 'iso_639_1': 'ru'...",Released,,Cinderella,False,0.803588,4.6,6,None


In [12]:
movies_df.to_json(
    r'C:\Users\tuanb\Downloads\Documents\Năm III - HKI\Đồ án 1\inserted_data_on_mongoDb_cloud\New Data\movies_with_backdrop_path.json',
    orient='records'
)